# API Documentation

## Authentication

### GET /api/auth/me

**Description:** Retrieves the authenticated user's information.
**Authentication:** Requires a valid `session-token` cookie.
**Request:**
- Method: GET
- Headers:
  - Cookie: `session-token=<token>`

**Response:**
- **200 OK**
  ```json
  {
    "success": true,
    "user": {
      "id": "user_id",
      "email": "user@example.com",
      "name": "User Name"
    }
  }
  ```
- **401 Unauthorized**
  ```json
  {
    "error": "Not authenticated"
  }
  ```
  ```json
  {
    "error": "Invalid session"
  }
  ```
- **500 Internal Server Error**
  ```json
  {
    "error": "Internal server error"
  }
  ```

**Test with cURL:**
```bash
curl -X GET http://localhost:3000/api/auth/me -H "Cookie: session-token=<token>"
```

### POST /api/auth/login

**Description:** Authenticates a user and returns a session token.
**Request:**
- Method: POST
- Body:
  ```json
  {
    "email": "user@example.com",
    "password": "user_password"
  }
  ```

**Response:**
- **200 OK**
  ```json
  {
    "success": true,
    "user": {
      "id": "user_id",
      "email": "user@example.com",
      "name": "User Name"
    },
    "token": "session_token"
  }
  ```
- **400 Bad Request**
  ```json
  {
    "error": "Email and password are required"
  }
  ```
- **401 Unauthorized**
  ```json
  {
    "error": "Invalid credentials"
  }
  ```
- **500 Internal Server Error**
  ```json
  {
    "error": "Internal server error"
  }
  ```

**Test with cURL:**
```bash
curl -X POST http://localhost:3000/api/auth/login -H "Content-Type: application/json" -d '{"email": "user@example.com", "password": "user_password"}'
```

### POST /api/auth/logout

**Description:** Logs out the user and invalidates the session token.
**Authentication:** Requires a valid `session-token` cookie.
**Request:**
- Method: POST

**Response:**
- **200 OK**
  ```json
  {
    "success": true
  }
  ```
- **500 Internal Server Error**
  ```json
  {
    "error": "Internal server error"
  }
  ```

**Test with cURL:**
```bash
curl -X POST http://localhost:3000/api/auth/logout -H "Cookie: session-token=<token>"
```

### POST /api/auth/register

**Description:** Registers a new user and returns a session token.
**Request:**
- Method: POST
- Body:
  ```json
  {
    "email": "user@example.com",
    "password": "user_password",
    "name": "User Name"
  }
  ```

**Response:**
- **200 OK**
  ```json
  {
    "success": true,
    "user": {
      "id": "user_id",
      "email": "user@example.com",
      "name": "User Name"
    },
    "token": "session_token"
  }
  ```
- **400 Bad Request**
  ```json
  {
    "error": "Email, password, and name are required"
  }
  ```
- **500 Internal Server Error**
  ```json
  {
    "error": "Internal server error"
  }
  ```

**Test with cURL:**
```bash
curl -X POST http://localhost:3000/api/auth/register -H "Content-Type: application/json" -d '{"email": "user@example.com", "password": "user_password", "name": "User Name"}'
```

## Leads

### POST /api/leads/enrich

**Description:** Enriches company data using a company name or website.
**Request:**
- Method: POST
- Body:
  ```json
  {
    "company": "Example Inc.",
    "website": "example.com"
  }
  ```

**Response:**
- **200 OK**
  ```json
  {
    "success": true,
    "data": {
      "name": "Example Inc.",
      "website": "example.com",
      "description": "...",
      "...": "..."
    },
    "timestamp": "2025-08-05T12:34:56.789Z"
  }
  ```
- **400 Bad Request**
  ```json
  {
    "error": "Company name or website is required"
  }
  ```
- **500 Internal Server Error**
  ```json
  {
    "error": "Failed to enrich company data"
  }
  ```

**Test with cURL:**
```bash
curl -X POST http://localhost:3000/api/leads/enrich -H "Content-Type: application/json" -d '{"company": "Example Inc.", "website": "example.com"}'
```

### POST /api/leads/score

**Description:** Scores a lead using a machine learning model, enhanced with real-time market data.
**Request:**
- Method: POST
- Body:
  ```json
  {
    "leadId": "lead_123",
    "industry": "SaaS",
    "...": "..."
  }
  ```

**Response:**
- **200 OK**
  ```json
  {
    "success": true,
    "score": 0.85,
    "marketData": {
      "...": "..."
    },
    "timestamp": "2025-08-05T12:34:56.789Z"
  }
  ```
- **500 Internal Server Error**
  ```json
  {
    "error": "Failed to score lead"
  }
  ```

**Test with cURL:**
```bash
curl -X POST http://localhost:3000/api/leads/score -H "Content-Type: application/json" -d '{"leadId": "lead_123", "industry": "SaaS"}'
```

### GET /api/leads

**Description:** Retrieves a paginated list of leads with optional filters.

**Authentication:** Requires a valid `session-token` cookie.

**Request:**
- Method: GET
- Query Parameters:
  - `page` (optional, default: 1): The page number to retrieve.
  - `limit` (optional, default: 10): The number of leads per page.
  - `industry` (optional): Filter by industry.
  - `location` (optional): Filter by location.
  - `scoreMin` (optional): Filter by minimum score.

**Response:**
- **200 OK**
  ```json
  {
    "leads": [
      {
        "id": "lead_123",
        "...": "..."
      }
    ],
    "pagination": {
      "page": 1,
      "limit": 10,
      "total": 100
    }
  }
  ```
- **401 Unauthorized**
  ```json
  {
    "error": "Authentication required"
  }
  ```
- **500 Internal Server Error**
  ```json
  {
    "error": "Failed to fetch leads"
  }
  ```

**Test with cURL:**
```bash
curl -X GET "http://localhost:3000/api/leads?page=1&limit=10&industry=SaaS" -H "Cookie: session-token=<token>"
```

### POST /api/leads

**Description:** Scrapes leads based on search parameters.

**Request:**
- Method: POST
- Body:
  ```json
  {
    "searchParams": {
      "...": "..."
    }
  }
  ```

**Response:**
- **200 OK**
  ```json
  {
    "success": true,
    "leads": [
      {
        "...": "..."
      }
    ],
    "timestamp": "2025-08-05T12:34:56.789Z"
  }
  ```
- **500 Internal Server Error**
  ```json
  {
    "error": "Failed to scrape leads"
  }
  ```

**Test with cURL:**
```bash
curl -X POST http://localhost:3000/api/leads -H "Content-Type: application/json" -d '{"searchParams": {}}'
```

## WebSocket

### GET /api/websocket

**Description:** Establishes a WebSocket connection.

**Authentication:** Requires a valid `session-token` cookie.

**Request:**
- Method: GET
- Query Parameters:
  - `clientId` (optional): The client ID to use for the connection. Defaults to the authenticated user's ID.

**Response:**
- **101 Switching Protocols**
- **401 Unauthorized**
- **500 Internal Server Error**

**Test with cURL:**
```bash
curl --include --no-buffer -H "Connection: Upgrade" -H "Upgrade: websocket" -H "Host: localhost:3000" -H "Origin: http://localhost:3000" -H "Sec-WebSocket-Key: $(openssl rand -base64 16)" -H "Sec-WebSocket-Version: 13" http://localhost:3000/api/websocket
```

## Analytics

### GET /api/analytics/realtime

**Description:** Retrieves real-time analytics data.

**Authentication:** Requires a valid `session-token` cookie.

**Request:**
- Method: GET
- Query Parameters:
  - `timeRange` (optional, default: "24h"): The time range for the analytics data.

**Response:**
- **200 OK**
  ```json
  {
    "success": true,
    "data": {
      "metrics": {
        "...": "..."
      },
      "trends": {
        "...": "..."
      },
      "alerts": [],
      "lastUpdated": "2025-08-05T12:34:56.789Z",
      "user": {
        "id": "user_id",
        "name": "User Name",
        "role": "admin"
      }
    }
  }
  ```
- **401 Unauthorized**
  ```json
  {
    "error": "Authentication required"
  }
  ```
- **500 Internal Server Error**
  ```json
  {
    "error": "Failed to fetch analytics"
  }
  ```

**Test with cURL:**
```bash
curl -X GET "http://localhost:3000/api/analytics/realtime?timeRange=24h" -H "Cookie: session-token=<token>"
```

## Admin

### GET /api/admin/users

**Description:** Retrieves a list of all users.

**Authentication:** Requires a valid `session-token` cookie with an `admin` role.

**Request:**
- Method: GET

**Response:**
- **200 OK**
  ```json
  {
    "success": true,
    "data": [
      {
        "id": "user_id",
        "email": "user@example.com",
        "name": "User Name",
        "role": "user"
      }
    ],
    "metadata": {
      "total": 1
    }
  }
  ```
- **401 Unauthorized**
  ```json
  {
    "error": "Authentication required"
  }
  ```
- **403 Forbidden**
  ```json
  {
    "error": "Admin access required"
  }
  ```
- **500 Internal Server Error**
  ```json
  {
    "error": "Internal server error"
  }
  ```

**Test with cURL:**
```bash
curl -X GET http://localhost:3000/api/admin/users -H "Cookie: session-token=<admin-token>"
```

### POST /api/admin/users

**Description:** Creates a new user.

**Authentication:** Requires a valid `session-token` cookie with an `admin` role.

**Request:**
- Method: POST
- Body:
  ```json
  {
    "email": "newuser@example.com",
    "password": "password123",
    "name": "New User",
    "role": "user"
  }
  ```

**Response:**
- **200 OK**
  ```json
  {
    "success": true,
    "data": {
      "id": "new_user_id",
      "email": "newuser@example.com",
      "name": "New User",
      "role": "user"
    }
  }
  ```
- **400 Bad Request**
  ```json
  {
    "error": "Email, password, and name are required"
  }
  ```
- **401 Unauthorized**
  ```json
  {
    "error": "Authentication required"
  }
  ```
- **403 Forbidden**
  ```json
  {
    "error": "Admin access required"
  }
  ```
- **500 Internal Server Error**
  ```json
  {
    "error": "Internal server error"
  }
  ```

**Test with cURL:**
```bash
curl -X POST http://localhost:3000/api/admin/users -H "Content-Type: application/json" -H "Cookie: session-token=<admin-token>" -d '{"email": "newuser@example.com", "password": "password123", "name": "New User", "role": "user"}'
```

## Enrichment

### POST /api/enrichment

**Description:** Enriches company data.
**Request:**
- Method: POST
- Body:
  ```json
  {
    "company": "Example Inc.",
    "website": "example.com"
  }
  ```

**Response:**
- **200 OK**
  ```json
  {
    "...": "..."
  }
  ```
- **500 Internal Server Error**
  ```json
  {
    "error": "Enrichment failed"
  }
  ```

**Test with cURL:**
```bash
curl -X POST http://localhost:3000/api/enrichment -H "Content-Type: application/json" -d '{"company": "Example Inc.", "website": "example.com"}'
```